In [294]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import numpy as np
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import SGDClassifier
import random
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.metrics import log_loss
from statsmodels.tools.tools import add_constant
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from statsmodels.genmod.generalized_linear_model import GLM
from utils import _add_constant,_hat_diag,_sigmoid_pred,_sigmoid_pred, _information_matrix, _predict, _predict_proba
#import R package brglm
base = importr('base')
d = {'package.dependencies': 'package_dot_dependencies',
     'package_dependencies': 'package_uscore_dependencies'}
brglm = importr('brglm',robject_translations=d)

In [295]:
#Load Data
features = [i.replace(' ', '_') for i in load_breast_cancer().feature_names.tolist()]

breast_cancer_df = pd.DataFrame(load_breast_cancer().data,columns=features)
target_df = pd.DataFrame(load_breast_cancer().target, columns=['y'])
X = breast_cancer_df.iloc[:,:5]
y = target_df

df = pd.concat([target_df,breast_cancer_df],axis=1).iloc[:,:6]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=0)

In [358]:
# Create small data set and rare event data set
num_successes_for_rare = int(((((1-df.y.mean())*df.shape[0])/.95)-((1-df.y.mean())*df.shape[0]))//1)
rare_inds = sorted(list(df[df.y==0].index) + random.sample(list(df[df.y==1].index),num_successes_for_rare))
# small_inds = random.sample(sorted(list(df.index)),50)


rare_df = df.iloc[rare_inds,:6]
rare_X = rare_df.drop('y',axis=1)
rare_y = rare_df['y']

separation_df = pd.read_csv('separation_df.csv',index_col=0).iloc[:,:6]
separation_X = separation_df.drop('y',axis=1)
separation_y = separation_df['y']

small_df = pd.read_csv('small_df.csv',index_col=0)
small_X = small_df.drop('y',axis=1)
small_y = small_df['y']


In [215]:
#Helper functions
def formula_from_df(df,y_var_name):
    features = list((df.drop(y_var_name,axis=1).columns))
    formula = y_var_name + '~' + ' + '.join(features)
    return formula

def Sigmoid_Pred(X, weights):
    z = np.dot(X,weights)
    sig =  1/(1 + np.exp(-1*z))
    sig = np.clip(sig,.000001,.999999)
    return sig

def hat_diag(X,weights):
    Xt = X.transpose()
    
    #Get diagonal of error
    y_pred = Sigmoid_Pred(X,weights)
    W = np.diag(y_pred*(1-y_pred))
   
    #Calculate Fisher Information Matrix
    I = np.linalg.multi_dot([Xt,W,X]) 

    #Get Diagonal of Hat Matrix
    hat = np.linalg.multi_dot([W**0.5,X,np.linalg.inv(I),Xt,W**0.5])
    hat_diag = np.diag(hat)
    return hat_diag

def marginal_effects(X,weights):
    #at means
    column_means = X.mean()
    p = Sigmoid_Pred(column_means,weights)
    at_means = np.ones(6)
    for i in range(weights.shape[0]):
        weights_copy = c.copy()
        weights_copy[i]+=1
        new_p =Sigmoid_Pred(means,weights_copy)
        at_means[i] = new_p-p
    
    #meaned
    averaged_marg_effs = np.ones((X.shape[0],X.shape[1]))
    for i in range(X.shape[0]):
        row = X.iloc[i]
        p = Sigmoid_Pred(row,c)
        for j in range(weights.shape[0]):
            weights_copy = weights.copy()
            weights_copy[j]+=1
            new_p = 1/(1+np.exp(-np.dot(row,weights_copy)))
            eff = new_p-p
            averaged_marg_effs[i,j] = eff
        ame = pd.DataFrame(averaged_marg_effs.mean(axis=0),index=X.columns, columns=['mean'])
        ame['at_means'] = at_means
    return ame

def information_matrix(X,weights):
    Xt = X.transpose()

    #Get diagonal of error
    y_pred = Sigmoid_Pred(X,weights)
    W = np.diag(y_pred*(1-y_pred))

    #Calculate Fisher Information Matrix
    I = np.linalg.multi_dot([Xt,W,X])
    return I

In [1335]:
#firth from scratch
def firth_logit(X,y,num_iter=10000,alpha=0.01,add_int=True):
    #add intercept if necessary
    if add_int==True:
        X = add_constant(X)
    
    #initialize weights
    weights=np.ones(X.shape[1])
    
    #Perform gradient descent
    for i in range(num_iter):
        y_pred = Sigmoid_Pred(X,weights)
        H = hat_diag(X,weights)
        #Update weights
        U = np.matmul((y -y_pred + H*(0.5 - y_pred)),X)
        weights += np.matmul(np.linalg.inv(I),U)*alpha
        if (i%1000==0):
            print('log likelihood: ',(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+0.5*np.log(np.linalg.det(I)))
    preds = Sigmoid_Pred(X,weights)
    weights = pd.Series(weights,index=X.columns)
    return preds, weights

In [1276]:
def firth_logit_r(df,y_var_name, formula, r_data=True, weights='none',all_coef_summary=False):
    #convert data frame to R df
    if r_data==False:
        if type(weights) ==str:
            with localconverter(ro.default_converter + pandas2ri.converter):
                df = ro.conversion.py2ri(df)
        else:
            with localconverter(ro.default_converter + pandas2ri.converter):
                df = ro.conversion.py2ri(df)
                weights = ro.vectors.FloatVector(weights)
    
    #create firth logit model
    if weights!='none':
        model = brglm.brglm(formula, data = df, family='binomial',pl=True, weights=weights)
    else:
        model = brglm.brglm(formula, data = df, family='binomial',pl=True)
    
    #extract coefficients
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    columns = list(df.colnames)
    columns[0]='Intercept'
    if all_coef_summary==True:
        coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns)
    else:
        coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns).Coef
    
    #get raw output and apply sigmoid
    preds = ro.r.predict(model,df)
    preds = 1/(1+np.exp(-np.array(preds)))       
    return preds, coefs

In [1300]:
#firth with hyperparameter
def tuned_firth_logit(X,y,lmbda=0.5,num_iter=10000,alpha=0.01,add_int=True):
    #add intercept if necessary
    if add_int==True:
        X = add_constant(X)
    
    #initialize weights
    weights=np.ones(X.shape[1])
    
    #Perform gradient descent
    for i in range(num_iter):
        y_pred = Sigmoid_Pred(X,weights)
        H = hat_diag(X,weights)
        U = np.matmul((y -y_pred + lmbda*H*(1 - 2*y_pred)),X)
        weights += np.matmul(np.linalg.inv(I),U)*alpha
        if (i%1000==0):
            print('log likelihood: ',(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+lmbda*np.log(np.linalg.det(I)))
    return pd.Series(weights,index=X.columns)

In [1224]:
def FLIC(X,y,num_iter=10000,lmbda=0.5):
    #get firth logit coefs
    coefs = tuned_firth_logit(X,y,lmbda=lmbda,num_iter=num_iter)
    
    #reestimate intercept
    coefs.drop('const',inplace=True)
    eta = np.dot(X,coefs)
    target = y-eta
    b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
    b0 = b0_model.params[0]
    coefs = pd.Series(b0,index=['Int']).append(coefs)
    
    #get predictions
    X=add_constant(X)
    preds = Sigmoid_Pred(X.values,coefs.values)
    return preds,coefs

In [213]:
def FLIC_brglm(df, y_var_name):
    formula = formula_from_df(df,y_var_name)
    with localconverter(ro.default_converter + pandas2ri.converter):
        df_r = ro.conversion.py2ri(df)
    model = brglm.brglm(formula, data = df_r, family='binomial',pl=True)
    summary = base.summary(model)
    summary_dic = {}
    for i in range(len(summary.names)):
        try:
            summary_dic[summary.names[i]]=pandas2ri.converter.ri2py(list(summary)[i])
        except:
            pass
    columns = list(firth_small_r.colnames)
    coefs = pd.DataFrame(summary_dic['coefficients'],columns=(['Coef','SE','Z','P']),index=columns).Coef[1:]
    y_var_name = formula.split('~')[0].strip()
    y = df[y_var_name].values
    X = df.drop(y_var_name,axis=1)
    eta = np.dot(X,coefs)
    target = y-eta
    b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
    b0 = pd.Series(b0_model.params[0],index=['Int'])
    coefs = b0.append(coefs)
    X=add_constant(X)
    preds = Sigmoid_Pred(X.values,coefs.values)
    return preds,coefs

In [1287]:
def FLAC(X,y):
    
    init_rows = df.shape[0]
    X = add_constant(X)
    
    
    #Build Hat Matrix = (W**0.5)*X*((XtWX)^-1)*Xt*W**0.5
    preds, weights = firth_logit_r(df=aug_df,
                                 y_var_name=y_var_name,
                                 formula=formula,
                                 r_data=False,
                                 weights=aug_sample_weights)
    weights = model.params
    H = hat_diag(X,weights)
    
    #Duplicate every row
    double_df = df.append(df)
    
    #Create a new copy of the original data
    pseudo_y_df = df
    #Change y to 1-y
    pseudo_y_df[y_var_name]=1-pseudo_y_df[y_var_name]
    
    #Append to doubled df
    aug_df = double_df.append(pseudo_y_df)
    
    #Create dummy for real vs. duplicated/pseudo data
    aug_df['real_data'] = 0
    aug_df['real_data'][init_rows:]=1
    
    
    #Create vector of weights = 1 for real data, hi/2 for augmentation data
    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),H/2,H/2]))
    
    
    #Get predictions and coefficients
    logit = LogisticRegression(solver='newton-cg',penalty='none')
    logit.fit(,y,sample_weights)
    model = sm.Logit(y,).fit()
    weights = model.params
    
    return preds, coefs
    

In [1282]:
def logF11(df,y_var_name,intercept=False):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #reseparate
    X = f_df.drop(y_var_name,axis=1)
    y = f_df[y_var_name]
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    model = sm.Logit(y,X).fit()
    coefs = model.params
    if intercept==True:
        eta = np.dot(X,coefs)
        target = y-eta
        b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
        b0 = pd.Series(b0_model.params[0],index=['Int'])
        coefs = b0.append(coefs)
        X=add_constant(X)

    preds = Sigmoid_Pred(X.values,coefs)
    return preds, coefs

Intermediate versions that may be worth keeping 

In [790]:
#log-F(1,1) just augmentation
def logF11_aug(df,y_var_name,R=False):
    '''Perform log-f(1,1) data augmentation
       Returns augmented df and observation weights'''
    
    num_rows = 2*(df.shape[1]-1)
    y_ind = df.columns.get_loc(y_var_name)
    
    aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))
    
    #augment y variable
    aug.iloc[range(0,num_rows,2),y_ind]=1
    y = aug[y_var_name]
    
    #augment X variables
    X = aug.drop(y_var_name,axis=1)
    for ind, rows in enumerate(range(0,X.shape[0],2)):
         X.iloc[rows:rows+2,ind]=1
    
    #bring it all together
    aug = pd.concat([y,X],axis=1)
    f_df = df.append(aug)
    
    #add offset
    f_df['real_data']=1
    f_df['real_data'][-aug.shape[0]:]=0
    
    #Calculate weights
    weights = f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)
    if R==True:
        with localconverter(ro.default_converter + pandas2ri.converter):
    
            f_df = ro.conversion.py2ri(f_df)
            weights = ro.vectors.FloatVector(weights)
    return f_df, weights      

In [787]:
#FLAC just augmentation
def FLAC_aug(df,y_var_name,R=False):
    '''Perform FLAC data augmentation
       Returns augmented df and observation weights'''
    
    init_rows = df.shape[0]
    X = add_constant(df.drop(y_var_name,axis=1))
    y = df[y_var_name]
    glm = GLM(y,X,family=sm.families.Binomial()).fit()
#     hat = glm.get_hat_matrix_diag()
    weights = glm.params
    y_pred = glm.predict(test_X)
    W = np.diag(y_pred*(1-y_pred))
    test_XtWtest_X = np.linalg.multi_dot([test_X.transpose(),W,test_X])
    I = np.linalg.inv(test_XtWtest_X)
    hat = np.diag(np.linalg.multi_dot([W**0.5,test_X,I,test_X.transpose(),W**0.5]))

    aug_df = pd.concat([df,df,df])
    aug_df[y_var_name][init_rows*2:]=1-aug_df[y_var_name][init_rows*2:]
    aug_df['pseudo_data'] = 0
    aug_df['pseudo_data'][init_rows:]=1

    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),hat/2,hat/2]))
    if R==True:
        with localconverter(ro.default_converter + pandas2ri.converter):
    
            aug_df = ro.conversion.py2ri(aug_df)
            aug_sample_weights = ro.vectors.FloatVector(aug_sample_weights)
    return aug_df, aug_sample_weights
    
    #Now run this through brglm

In [ ]:
ro.r

In [212]:
np.array(ro.vectors.FloatVector(np.array([1,2,3])))

array([1., 2., 3.])

In [785]:
#created augmented datasets

#log-f(1,1)
# all_f, all_weights_f = logf11_aug(all_df,'y')
small_f, small_weights_f = logf11_aug(small_df,'y')
rare_f, rare_weights_f = logf11_aug(rare_df,'y')
separation_f, separation_weights_f = logf11_aug(separation_df,'y')

#FLAC
# all_FLAC, all_weights_FLAC = FLAC_aug(all_df,'y')
small_FLAC, small_weights_FLAC = FLAC_aug(small_df,'y')
rare_FLAC, rare_weights_FLAC = FLAC_aug(rare_df,'y')
separation_FLAC, separation_weights_FLAC = FLAC_aug(separation_df,'y')

#created R augmented datasets

#log-f(1,1)
# all_f, all_weights_f = logf11_aug(all_df,'y')
small_f_r, small_weights_f_r = logf11_aug(small_df,'y')
rare_f_r, rare_weights_f_r = logf11_aug(rare_df,'y')
separation_f_r, separation_weights_f_r = logf11_aug(separation_df,'y')

#FLAC
# all_FLAC, all_weights_FLAC = FLAC_aug(all_df,'y')
small_FLAC_r, small_weights_FLAC_r = FLAC_aug(small_df,'y')
rare_FLAC_r, rare_weights_FLAC_r = FLAC_aug(rare_df,'y')
separation_FLAC_r, separation_weights_FLAC_r = FLAC_aug(separation_df,'y')

/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/remycanario/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set o

WIP

In [ ]:
def boosted_firth(X,y,num_steps=500):
    init_rows = X.shape[0]
    aug_X = X.append(X)
    aug_y = y.append(1-y)
    eta0=np.full((1,,sm.Logit(aug_y,aug_X.const).fit().params[0])
    weights = np.ones(aug_X.shape[1])
    for i in range(num_steps):
        hat_diag = hat(aug_X,weights)
        y_pred = Sigmoid_Pred(aug_X,weights)
        offset = (1+h/2)*(aug_y-y_pred*())

In [1325]:
def testing(df,y_var_name,X_train,y_train,X_test,y_test):
    models = ['logit','l2','firth','FLIC','FLAC','log-f(1,1)','log-f(1,1)_with_int']

    c_X_train = add_constant(X_train)
    c_X_test = add_constant(X_test)
    
    base = sm.Logit(y_train,c_X_train).fit()
    control_proba = base.predict(c_X_test)
    control_preds = control_proba.round()
    
    l2 = LogisticRegression()
    l2.fit(X_train,y_train)
    l2_proba = l2.predict_proba(X_test)
    l2_preds = l2.predict(X_test)
    
    firth_proba, firth_coefs = firth_logit(X_train,y_train)
    firth_preds = firth_proba.round()
    
    FLIC_proba, FLIC_coefs = FLIC(X_train,y_train)
    FLIC_preds = FLIC_proba.round()
    
    FLAC_proba, FLAC_coefs = FLAC_brglm(df,y_var_name)
    FLAC_preds = FLAC_proba.round()
    
    logF11_proba, logF11_coefs = logF11(df,y_var_name)
    logF11_preds = logF11_proba.round()
    
    logF11_int_proba, logF11_coefs = logF11(df,y_var_name,intercept=True)
    logF11_int_preds = logF11_int_round()
    
    proba = [control_proba, l2_proba, firth_proba, FLIC_proba, FLAC_proba, logF11_proba, logF11_int_proba]
    preds = [control_preds, l2_preds, firth_preds, FLIC_preds, FLAC_preds, logF11_preds, logF11_int_preds]
    return proba, preds
    
    

In [ ]:
def FLAC(X,y):
    
    init_rows = df.shape[0]
    X = add_constant(X)
    
    
    #Build Hat Matrix = (W**0.5)*X*((XtWX)^-1)*Xt*W**0.5
    preds, weights = firth_logit_r(df=aug_df,
                                 y_var_name=y_var_name,
                                 formula=formula,
                                 r_data=False,
                                 weights=aug_sample_weights)
    weights = model.params
    H = hat_diag(X,weights)
    
    #Duplicate every row
    double_df = df.append(df)
    
    #Create a new copy of the original data
    pseudo_y_df = df
    #Change y to 1-y
    pseudo_y_df[y_var_name]=1-pseudo_y_df[y_var_name]
    
    #Append to doubled df
    aug_df = double_df.append(pseudo_y_df)
    
    #Create dummy for real vs. duplicated/pseudo data
    aug_df['real_data'] = 0
    aug_df['real_data'][init_rows:]=1
    
    
    #Create vector of weights = 1 for real data, hi/2 for augmentation data
    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),H/2,H/2]))
    
    
    #Get predictions and coefficients
    logit = LogisticRegression(solver='newton-cg',penalty='none')
    logit.fit(,y,sample_weights)
    model = sm.Logit(y,).fit()
    weights = model.params
    
    return preds, coefs
    

In [225]:
sklogit = LogisticRegression(solver='newton-cg',penalty='none',fit_intercept=False)
sklogit.fit(small_X,small_y)
                

/Users/remycanario/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

In [260]:
type(pd.Series(sklogit.coef_.flatten()))

pandas.core.series.Series

In [384]:
#Firth Class
#Things left to do: confidence intervals, Cauchy, generalize marginal effects
class PMLE():
    class Firth_Logit():
        def __init__(self,num_iters=10000, alpha=0.01,add_int=True,lmbda=0.5,FLAC=False, FLIC=False):

            self.alpha = alpha
            self.num_iters = num_iters
            self.add_int = add_int
            self.lmbda=lmbda
            self.FLAC = FLAC
            self.FLIC=FLIC
        
        def firth_gd(self,X,y,weights):
            y_pred = _sigmoid_pred(X=X,weights=weights)
            H =_hat_diag(X,weights)
            I = _information_matrix(X,weights)
            U = np.matmul((y -y_pred + self.lmbda*H*(1 - 2*y_pred)),X)
            weights += np.matmul(np.linalg.inv(I),U)*self.alpha
            return weights
        
        def fit(self,X,y):
            #add intercept if necessary
            orig_X = X
            if self.add_int==True:
                X =_add_constant(X)
            self.X = X
            self.y = y

            #initialize weights
            weights=np.ones(X.shape[1])
            
            
            #Perform gradient descent
            for i in range(self.num_iters):
                weights = self.firth_gd(X,y,weights)
            
            if (self.FLAC==True)&(self.FLIC==True):
                X,y,aug_sample_weights=_FLAC_aug(X,y,weights)
                self.X = X
                self.y = y
                sklogit = LogisticRegression(solver='newton-cg',penalty='none',fit_intercept=False)
                sklogit.fit(X,y,sample_weight=aug_sample_weights)
                weights = sklogit.coef_[1:]
                eta = np.dot(orig_X,weights)
                target = y-eta
                b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
                b0 = b0_model.params[0]
                weights = np.insert(weights,0,b0)
            
            elif self.FLAC==True:
                X,y,aug_sample_weights=_FLAC_aug(X,y,weights)
                self.X = X
                self.y = y
                sklogit = LogisticRegression(solver='newton-cg',penalty='none',fit_intercept=False)
                sklogit.fit(X,y,sample_weight=aug_sample_weights)
                weights = sklogit.coef_
                
            
            elif self.FLIC==True:
                weights = weights[1:]
                eta = np.dot(orig_X,weights)
                target = y-eta
                b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
                b0 = b0_model.params[0]
                weights = np.insert(weights,0,b0)
            
            weights = pd.Series(weights.flatten(),index=self.X.columns)
            self.weights = weights
            
            I = _information_matrix(X,weights)
            hat_matrix_diag = _hat_diag(X,weights)
            Hessian = -I
            y_pred = _sigmoid_pred(X,weights)
            
            self.I = I
            self.hat_matrix_diag = hat_matrix_diag
            self.Hessian = Hessian
            
            
            self.log_likelihood = (y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).sum()+0.5*np.log(np.linalg.det(I))
            
        def predict(self,X):
            if self.FLAC==True:
                X = _FLAC_pred_aug(X)
            if X.shape[1]==self.X.shape[1]-1:
                X=_add_constant(X)
            return _predict(X,self.weights)
        
        def predict_proba(self,X):
            if self.FLAC==True:
                X = _FLAC_pred_aug(X)
            if X.shape[1]==self.X.shape[1]-1:
                X=_add_constant(X)
            return _predict_proba(X,self.weights)
        
        
        def marginal_effects(self,values=None):
                
            def at_specific_values(self,values):
                n_features = self.weights.shape[0]
                if values.shape[0]==n_features-1:
                    values = _add_constant(values)
                
                p = _sigmoid_pred(values,self.weights)
                effs = np.ones(n_features)
                for i in range(n_features):
                    weights_copy = self.weights.copy()
                    weights_copy[i]+=1
                    new_p =_sigmoid_pred(values,weights_copy)
                    effs[i] = new_p-p
                return effs
            
            #at means
            column_means = self.X.mean()
            at_means = at_specific_values(weights=column_means)

            #meaned
            averaged_marg_effs = np.ones((self.X.shape[0],self.X.shape[1]))
            for i in range(self.X.shape[0]):
                row = self.X.iloc[i]
                p = _sigmoid_pred(row,self.weights)
                for j in range(self.weights.shape[0]):
                    weights_copy = self.weights.copy()
                    weights_copy[j]+=1
                    new_p =_sigmoid_pred(row,weights_copy)
                    eff = new_p-p
                    averaged_marg_effs[i,j] = eff
                ame = pd.DataFrame(averaged_marg_effs.mean(axis=0),index=self.X.columns, columns=['mean'])
                ame['at_means'] = at_means
            #user requested
            if (type(values)==numpy.ndarray) | (type(values)==pandas.core.series.Series):
                user_requested = at_specific_values(values)
                ame['requested_values'] = user_requsted
            return ame
        
        
    
    class logF11():
        def __init__(self,intercept=False):
            self.intercept=False
        
        def data_augementation(self,df,y_var_name):
            num_rows = 2*(df.shape[1]-1)
            y_ind = df.columns.get_loc(y_var_name)

            aug = pd.DataFrame(0,columns=df.columns,index=(range(num_rows)))

            #augment y variable
            aug.iloc[range(0,num_rows,2),y_ind]=1
            y = aug[y_var_name]

            #augment X variables
            X = aug.drop(y_var_name,axis=1)
            for ind, rows in enumerate(range(0,X.shape[0],2)):
                 X.iloc[rows:rows+2,ind]=1

            #bring it all together
            aug = pd.concat([y,X],axis=1)
            f_df = df.append(aug)

            #add offset
            f_df['real_data']=1
            f_df['real_data'][-aug.shape[0]:]=0
            f_df['real_data'].apply(lambda x: 0.5 if x == 0 else 1)

            #reseparate
            X = f_df.drop(y_var_name,axis=1)
            y = f_df[y_var_name]
            
            self.X = X
            self.y = y
    
            return X, y
        
        def fit(self,df,y_var_name):
            X, y = self.data_augementation(df,y_var_name)
            model = sm.Logit(y,X).fit()
            weights = model.params
            if self.intercept==True:
                eta = np.dot(X,weights)
                target = y-eta
                b0_model = sm.OLS(target,np.ones(y.shape[0])).fit()
                b0 = b0_model.params[0]
                weights = np.insert(weights,0,b0)
                X = _add_constant(X)
            self.X = X
            weights = pd.Series(weights,index=X.columns)
            self.weights = weights
        
        
        
        def predict(self,X):
            return _predict(X,self.weights)
        
        def predict_proba(self,X):
            return _predict_proba(X,self.weights)
            



In [298]:
f = PMLE.Firth_Logit(num_iters=5000,FLAC=False)

In [299]:
f.fit(small_X,small_y)

/Users/remycanario/Documents/DataScience/wip-pmle/utils.py:18: RuntimeWarning: overflow encountered in exp
  sig =  1/(1 + np.exp(-1*z))


In [122]:
def weighted(X,y,weights,sample_weights):
    one = sample_weights * (y*_sigmoid_pred(X,weights)-1) * y
    two = np.dot(X.transpose(),one)
    return two

In [ ]:
out = -np.sum(sample_weight * log_logistic(yz)) + .5 * alpha * np.dot(w, w)

z = expit(yz)
z0 = sample_weight * (z - 1) * y

grad[:n_features] = safe_sparse_dot(X.T, z0) + alpha * w


In [ ]:
z  = np.expit(y*np.dot(X,weights))
z0 = sample_weights * (z-1) * y

In [142]:
def test(X,weights):
    z = np.dot(X,weights)
    hessian = ((X**2)*np.exp(z))/(1+np.exp(z))**2
    return hessian

In [347]:
num_iters = 5000
eta = np.dot(small_X,f.weights[1:])
b0 = 1
X0 = np.ones((small_X.shape[0],1))
for i in range(num_iters):
    y_pred = FLIC_sigmoid_pred(X0,b0,eta)
    W = np.diag(y_pred*(1-y_pred))
    I = np.linalg.multi_dot([X0.transpose(),W,X0])
    U = small_y-y_pred
    b0 += (1/I)*U*0.01
    

LinAlgError: 1-dimensional array given. Array must be two-dimensional

In [307]:
def FLIC_sigmoid_pred(X, weights,eta):
    z = np.dot(X,weights)
    sig =  1/(1 + np.exp(-z-eta))
    sig = np.clip(sig,.000001,.999999)
    return sig

def _information_matrix(X,weights,eta):
    Xt = X.transpose()

    #Get diagonal of error
    y_pred = FLIC_sigmoid_pred(X,weights,eta)
    W = np.diag(y_pred*(1-y_pred))

    #Calculate Fisher Information Matrix
    I = np.linalg.multi_dot([Xt,W,X])
    return I
def U =

def FLIC_two(X,y,weights):
    
    

In [362]:
c_X_train = add_constant(X_train)
c_X_test = add_constant(X_test)

base = sm.Logit(y_train,c_X_train).fit()
control_proba = base.predict(c_X_test)
control_preds = control_proba.round()




Optimization terminated successfully.
         Current function value: 0.148958
         Iterations 10


/Users/remycanario/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [388]:
from sklearn.metrics import roc_auc_score

In [ ]:
ind = ['small','rare','separation']
small_results = pd.DataFrame
rare_results = pd.DataFrame
separation_results = pd.DataFrame

In [390]:
roc_auc_score(y_test,small_control_proba)

0.02826294061606858

In [403]:
small_l2_proba[

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
       1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.])

In [401]:
small_l2_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0])

In [410]:
accuracy_score(y_test,small_firth_proba.round())

0.08771929824561403

In [459]:
labels = ['Logit', 'L2','Firth','FLIC','FLAC','FLICFLAC']
results = [rare_control_proba, rare_l2_proba, rare_firth_proba,rare_FLIC_proba,rare_FLAC_proba[:114], rare_FLICFLAC_proba[:114]]

metrics = []
for result in results:
    acc = accuracy_score(y_test,result.round())
    bin_cross_entr = log_loss(y_test,result)
    roc_auc = roc_auc_score(y_test,result)
    metrics.append([acc,bin_cross_entr,roc_auc])

rare_results = pd.DataFrame.from_dict(dict(zip(labels,metrics)))
rare_results.index = ['Accuracy_Score','Binary_Cross_Entropy','ROC_AUC']
rare_results = rare_results.transpose()


NameError: name 'rare_FLICFLAC_proba' is not defined

In [453]:
labels = ['Logit', 'L2','Firth','FLIC','FLAC','FLICFLAC']
results = [separation_control_proba, separation_l2_proba, 
           separation_firth_proba,separation_FLIC_proba,separation_FLAC_proba[:114],separation_FLICFLAC_proba[:114]]

metrics = []
for result in results:
    acc = accuracy_score(y_test,result.round())
    bin_cross_entr = log_loss(y_test,result)
    roc_auc = roc_auc_score(y_test,result)
    metrics.append([acc,bin_cross_entr,roc_auc])

separation_results = pd.DataFrame.from_dict(dict(zip(labels,metrics)))
separation_results.index = ['Accuracy_Score','Binary_Cross_Entropy','ROC_AUC']
separation_results = separation_results.transpose()


In [454]:
separation_results

,Accuracy_Score,Binary_Cross_Entropy,ROC_AUC
Logit,0.938596,0.247381,0.988885
L2,0.912281,0.241874,0.962210
Firth,0.921053,0.212465,0.973007
FLIC,0.842105,0.251296,0.973007
FLAC,0.921053,0.211935,0.973007


In [440]:
labels = ['Logit', 'L2','Firth','FLIC','FLAC','FLICFLAC']
results = [small_control_proba, small_l2_proba, small_firth_proba,small_FLIC_proba,small_FLAC_proba[:114],small_FLICFLAC_proba[:114]]

metrics = []
for result in results:
    acc = accuracy_score(y_test,result.round())
    bin_cross_entr = log_loss(y_test,result)
    roc_auc = roc_auc_score(y_test,result)
    metrics.append([acc,bin_cross_entr,roc_auc])

small_results = pd.DataFrame.from_dict(dict(zip(labels,metrics)))
small_results.index = ['Accuracy_Score','Binary_Cross_Entropy','ROC_AUC']
small_results = small_results.transpose()


1
1
1
1
1


In [455]:
rare_results

,Accuracy_Score,Binary_Cross_Entropy,ROC_AUC
Logit,0.789474,1.156788,0.971737
L2,0.614035,0.877821,0.969197
Firth,0.771930,0.787551,0.953636
FLIC,0.824561,0.497928,0.953636
FLAC,0.745614,0.818522,0.955224


In [456]:
small_results

,Accuracy_Score,Binary_Cross_Entropy,ROC_AUC
Logit,0.078947,6.210036,0.028263
L2,0.105263,2.341475,0.031439
Firth,0.087719,3.814560,0.033026
FLIC,0.149123,3.660701,0.033026
FLAC,0.087719,3.798521,0.033026


In [457]:
separation_results

,Accuracy_Score,Binary_Cross_Entropy,ROC_AUC
Logit,0.938596,0.247381,0.988885
L2,0.912281,0.241874,0.962210
Firth,0.921053,0.212465,0.973007
FLIC,0.842105,0.251296,0.973007
FLAC,0.921053,0.211935,0.973007


In [450]:
#MLE
base = sm.Logit(small_y,add_constant(small_X)).fit()
small_control_proba = base.predict(add_constant(X_test))
small_control_preds = control_proba.round()

base = sm.Logit(rare_y,add_constant(rare_X)).fit()
rare_control_proba = base.predict(add_constant(X_test))
rare_control_preds = control_proba.round()

try:
    base = sm.Logit(separation_y,add_constant(separation_X)).fit()
    separation_control_proba = base.predict(add_constant(X_test))
    separation_control_preds = control_proba.round()
except:
    separation_control_proba = 'NA'
    separation_control_preds = 'NA'

#L2
l2 = LogisticRegression()

l2.fit(small_X,small_y)
small_l2_proba = l2.predict_proba(X_test)[:,1]
small_l2_preds = l2.predict(X_test)

l2.fit(rare_X,rare_y)
rare_l2_proba = l2.predict_proba(X_test)[:,1]
rare_l2_preds = l2.predict(X_test)

l2.fit(separation_X,separation_y)
separation_l2_proba = l2.predict_proba(X_test)[:,1]
separation_l2_preds = l2.predict(X_test)

# #Firth
# firth = PMLE.Firth_Logit(num_iters=5000)

# firth.fit(small_X,small_y)
# small_firth_proba = firth.predict_proba(X_test)
# small_firth_preds = firth.predict(X_test)

# firth.fit(rare_X,rare_y)
# rare_firth_proba = firth.predict_proba(X_test)
# rare_firth_preds = firth.predict(X_test)

# firth.fit(separation_X,separation_y)
# separation_firth_proba = firth.predict_proba(X_test)
# separation_firth_preds = firth.predict(X_test)

# #FLIC
# FLIC = PMLE.Firth_Logit(num_iters=5000,FLIC=True)
# FLIC.fit(small_X,small_y)
# small_FLIC_proba = FLIC.predict_proba(X_test)
# small_FLIC_preds = FLIC.predict(X_test)

# FLIC.fit(rare_X,rare_y)
# rare_FLIC_proba = FLIC.predict_proba(X_test)
# rare_FLIC_preds = FLIC.predict(X_test)

# FLIC.fit(separation_X,separation_y)
# separation_FLIC_proba = FLIC.predict_proba(X_test)
# separation_FLIC_preds = FLIC.predict(X_test)

# #FLAC
# FLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True)
# FLAC.fit(small_X,small_y)
# small_FLAC_proba = FLAC.predict_proba(X_test)
# small_FLAC_preds = FLAC.predict(X_test)

# FLAC.fit(rare_X,rare_y)
# rare_FLAC_proba = FLAC.predict_proba(X_test)
# rare_FLAC_preds = FLAC.predict(X_test)

# FLAC.fit(separation_X,separation_y)
# separation_FLAC_proba = FLAC.predict_proba(X_test)
# separation_FLAC_preds = FLAC.predict(X_test)

Optimization terminated successfully.
         Current function value: 0.199704
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.027104
         Iterations 15
Optimization terminated successfully.
         Current function value: 0.104328
         Iterations 14


/Users/remycanario/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/remycanario/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/remycanario/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/remycanario/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [414]:
FLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True)
FLAC.fit(small_X,small_y)
small_FLAC_proba = FLAC.predict_proba(X_test)
small_FLAC_preds = FLAC.predict(X_test)


/Users/remycanario/Documents/DataScience/wip-pmle/utils.py:18: RuntimeWarning: overflow encountered in exp
  sig =  1/(1 + np.exp(-1*z))
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [458]:
FLICFLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True,FLIC=True)
FLAC.fit(small_X,small_y)
small_FLICFLAC_proba = FLICFLAC.predict_proba(X_test)
small_FLICFLAC_preds = FLICFLAC.predict(X_test)
FLICFLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True,FLIC=True)
FLAC.fit(rare_X,rare_y)
rare_FLICFLAC_proba = FLICFLAC.predict_proba(X_test)
rare_FLICFLAC_preds = FLICFLAC.predict(X_test)
FLICFLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True,FLIC=True)
FLAC.fit(separation_X,separation_y)
separation_FLICFLAC_proba = FLICFLAC.predict_proba(X_test)
separation_FLICFLAC_preds = FLICFLAC.predict(X_test)

/Users/remycanario/Documents/DataScience/wip-pmle/utils.py:18: RuntimeWarning: overflow encountered in exp
  sig =  1/(1 + np.exp(-1*z))
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/remycanario/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AttributeError: 'Firth_Logit' object has no attribute 'X'

In [415]:
FLICFLAC = PMLE.Firth_Logit(num_iters=5000,FLAC=True,FLIC=True)
FLAC.fit(separation_X,separation_y)
separation_FLICFLAC_proba = FLICFLAC.predict_proba(X_test)
separation_FLICFLAC_preds = FLICFLAC.predict(X_test)

(342,)

In [ ]:
def FLAC_aug(X,y,weights):
    init_rows = X.shape[0]
    hat_diag = _hat_diag(X,weights)
    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),hat_diag/2,hat_diag/2]))

    X = X.append(X).append(X)
    X['pseudo_data']=0
    X['pseudo_data'][init_rows:]=1
    y = y.append(y).append(1-y)
    return X, y, aug_sample_weights

In [376]:
def FLAC_aug(X,weights,y=None):
    init_rows = X.shape[0]
    hat_diag = _hat_diag(X,weights)
    aug_sample_weights = pd.Series(np.concatenate([np.ones(init_rows),hat_diag/2,hat_diag/2]))

    X = X.append(X).append(X)
    X['pseudo_data']=0
    X['pseudo_data'][init_rows:]=1
    if isinstance(y, np.ndarray) | isinstance(y, pd.DataFrame):
       
        y = y.append(y).append(1-y)
    return X, y, aug_sample_weights
    
                

In [ ]:
sklogit = LogisticRegression(solver='newton-cg',penalty='none',fit_intercept=False)
    sklogit.fit(X,y,sample_weight=aug_sample_weights)
    weights = sklogit.coef_